In [6]:
import torch
from torch import nn

# 一、填充(Padding)
"""
在应用多层卷积时,常常丢失边缘像素。由于通常使用小卷积核,因此对于任何单个卷积,
可能只会丢失几个像素。但随着应用许多连续卷积层,累积丢失的像素数就多了。
解决这个问题的简单方法即为填充:在输入图像的边界填充元素(通常填充元素是0)

不使用填充时,输入形状为h*w,卷积核形状为kh*kw,输出形状为 (h-kh+1)*(w-kw+1)
使用填充时,如果添加ph行填充(大约一半在顶部,一半在底部)和pw列填充(左侧大约一半,右侧一半),
则输出形状将为 (h-kh+ph+1)*(w-kw+pw+1) 即输出的高度和宽度将分别增加ph和pw。

在许多情况下,设置 ph = kh - 1和pw = kw - 1 ,使输入和输出具有相同的高度和宽度
这样可以在构建网络时更容易地预测每个图层的输出形状

卷积神经网络中卷积核的高度和宽度通常为奇数,例如1、3、5或7。
选择奇数的好处是,保持空间维度的同时,可以在顶部和底部填充相同数量的行,在左侧和右侧填充相同数量的列

此外,使用奇数的核大小和填充大小也提供了书写上的便利。对于任何二维张量X, 当满足:
1. 卷积核的大小是奇数
2. 所有边的填充行数和列数相同
3. 输出与输入具有相同高度和宽度
则可以得出:输出Y[i, j]是通过以输入X[i, j]为中心,与卷积核进行互相关计算得到的。
"""
# 为了方便起见,我们定义了一个计算卷积层的函数。
# 此函数初始化卷积层权重,并对输入和输出提高和缩减相应的维数
def comp_conv2d(conv2d, X):
    # 这里的（1,1）表示批量大小和通道数都是1
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # 省略前两个维度：批量大小和通道
    return Y.reshape(Y.shape[2:])
# 注意,这里每边都填充了1行或1列,因此总共添加了2行或2列
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
X = torch.rand(size=(8,8))
print(f"设置卷积核为3*3,padding=(2,2)的Y的形状:{comp_conv2d(conv2d, X).shape}")
conv2d = nn.Conv2d(1, 1, kernel_size=(5,3), padding=(2,1)) # 当卷积核的高度和宽度不同时,我们可以填充不同的高度和宽度,使输出和输入具有相同的高度和宽度
print(f"设置卷积核为5*3,padding=(4,2)的Y的形状:{comp_conv2d(conv2d, X).shape}")

# 二、步幅(Stride)
"""
在计算互相关时,卷积窗口从输入张量的左上角开始,向下、向右滑动。在前面的例子中,默认每次滑动
一个元素。但是,有时候为了高效计算或是缩减采样次数,卷积窗口可以跳过中间位置,每次滑动多个元素。
每次滑动的元素数量称为步幅,步幅可以在垂直方向h和水平方向w具有不同的大小

垂直步幅为sh,水平步幅为sw时, 输出形状为:
floor( (h-kh+ph+sh)/sh ) * ( (w-kw+pw+sw)/sw )
如果输入的高度和宽度可以被垂直步幅和水平步幅整除,输出形状为:
h/sh * w/sw
"""

"""
为了简洁起见,当输入高度和宽度两侧的填充数量分别为ph和pw时,我们称之为填充(ph, pw)。当ph = pw =
p时,填充是p。同理,当高度和宽度上的步幅分别为sh和sw时,我们称之为步幅(sh, sw)。特别地,当sh = sw = s时,
我们称步幅为s。默认情况下,填充为0,步幅为1。在实践中,我们很少使用不一致的步幅或填充,也就是说,
我们通常有ph = pw和sh = sw。
"""
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2) # 将高度和宽度的步幅设置为2,从而将输入的高度和宽度减半
print(f"设置卷积核为3*3,padding=(2,2),stride=2的Y的形状:{comp_conv2d(conv2d, X).shape}")

设置卷积核为3*3,padding=(2,2)的Y的形状:torch.Size([8, 8])
设置卷积核为5*3,padding=(4,2)的Y的形状:torch.Size([8, 8])
设置卷积核为3*3,padding=(2,2),stride=2的Y的形状:torch.Size([4, 4])
